# Fit emimTF2N data

EoS: PR

Parameter T dependence: Linear

N (total fitting parameters): 4

## Import Functions

In [1]:
import idaes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

import sys
sys.path.append('../../../')

from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.opt import TerminationCondition, SolverStatus

from idaes.core import FlowsheetBlock
import idaes.logger as idaeslog
# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
# Import unit models from the model library
from idaes.generic_models.unit_models import Flash
# Import degrees of freedom tool
from idaes.core.util.model_statistics import degrees_of_freedom

# parmest (binary_param2)
from bip_fitting_functions import linear

import scipy.stats as stats

## Load Data

In [2]:
data_full = pd.read_csv('../r125_emimtf2n_full.csv')
data_subset = pd.read_csv('../r125_emimtf2n_subset.csv')
data_extended = pd.read_csv('../r125-emimtf2n-extended.csv')

## Load Configuration + Check DOF + Load Parameters

In [3]:
lhs_results1 = pd.read_csv('../Data/Fits/LHS_Fits/PR_linTdep_LHS.csv',index_col=0)
lhs_results1 = lhs_results1[['sc_param1','sc_param2','sc_param3','sc_param4','SSR']]
lhs_results2 = pd.read_csv('Data/Init_Final/LHS_Output/PR_linTdep_LHS_final.csv',index_col=0)
lhs_results2 = lhs_results2[['sc_param1','sc_param2','sc_param3','sc_param4','SSR']]

In [4]:
lhs_results = pd.concat([lhs_results1,lhs_results2],axis=0).reset_index(drop=True)

In [5]:
lhs_results.dropna(axis=0)

,sc_param1,sc_param2,sc_param3,sc_param4,SSR
24,-0.920014,-0.286915,0.621217,0.255704,0.117352
71,-0.115629,-0.834497,-0.784946,0.781298,0.117352
150,-0.752131,-0.477515,-0.363124,0.407035,0.119971
473,0.556070,-1.038849,-0.821634,1.088539,0.117352
509,-0.403903,-0.505431,-0.798019,0.472897,0.117352
1224,-0.332046,-0.166957,-0.091635,0.142481,0.117808
1249,0.107046,-1.351754,-0.828205,1.373039,0.117352
1669,0.534416,-0.861035,-1.269344,0.855259,0.117352
2379,1.182459,-1.868503,-1.462754,1.734388,5.369137
2468,-1.421433,-1.791666,0.552875,1.786227,0.117352


In [6]:
lhs_best = lhs_results[lhs_results.SSR == lhs_results.SSR.min()]
lhs_best

,sc_param1,sc_param2,sc_param3,sc_param4,SSR
4504,-0.73579,-1.504159,0.132012,1.491727,0.117352


In [7]:
sc_param1 = lhs_best.at[4504, 'sc_param1']
sc_param2 = lhs_best.at[4504, 'sc_param2']
sc_param3 = lhs_best.at[4504, 'sc_param3']
sc_param4 = lhs_best.at[4504, 'sc_param4']

In [8]:
from hfc125_emimtf2n_PR import configuration 

In [9]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.properties = GenericParameterBlock(default=configuration)
m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})
DOF_initial = degrees_of_freedom(m)
print('The initial DOF is {0}'.format(DOF_initial))

The initial DOF is 7


## Fit Parameters

In [10]:
parameters, obj_value, a = linear(data_subset, configuration, 'R125', 'emimTf2N', "x_R125", "x_emimTf2N", 
        init_temp =  283.1, init_press =   399300 , init_x_c1 =    0.448, init_x_c2 = 0.552,
        init_kappa_2_1A = sc_param1, init_kappa_1_2A = sc_param2,
        init_kappa_2_1B = sc_param3, init_kappa_1_2B = sc_param4, 
        eps = 0.1, scaling_fac = 1e-9, 
        filename='Data/Fits/Ipopt_Output/PR_linTdep_final.txt')

2022-11-22 13:21:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-11-22 13:21:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-11-22 13:21:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-11-22 13:21:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-11-22 13:21:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-11-22 13:21:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-11-22 13:21:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-11-22 13:21:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-11-22 13:21:38 [INFO] idaes

/Users/bridgettebefort/opt/anaconda3/envs/idaes-pse-dev/lib/python3.9/site-packages/pyomo/contrib/interior_point/interface.py:418: RuntimeWarning: divide by zero encountered in true_divide
  data = (duals_primals_lb/(primals - self._nlp.primals_lb()) +


In [11]:
print('Objective (SSE):',obj_value)

Objective (SSE): 0.11735158519179448


In [12]:
print('Parameters:',parameters)

Parameters: fs.properties.PR_kappa_A[R125,emimTf2N]   -0.052388
fs.properties.PR_kappa_A[emimTf2N,R125]   -0.102865
fs.properties.PR_kappa_B[R125,emimTf2N]    0.032664
fs.properties.PR_kappa_B[emimTf2N,R125]    0.042199
dtype: float64


In [13]:
params2save = np.zeros(8)
params2save[0] = parameters[0]
params2save[1] = parameters[1]
params2save[2] = parameters[2]
params2save[3] = parameters[3]

In [14]:
np.savetxt('Data/Fits/PR_params_linTdep.csv',params2save,delimiter=',')

In [15]:
cov = np.array(a)
print('Covariance Matrix:')
print(cov)

Covariance Matrix:
[[ 9.81317823e-05  8.11102893e-04 -9.07602970e-05 -7.76123043e-04]
 [ 8.11102893e-04  2.66089340e-02 -8.59875064e-04 -2.75275645e-02]
 [-9.07602970e-05 -8.59875064e-04  8.49454636e-05  8.37388072e-04]
 [-7.76123043e-04 -2.75275645e-02  8.37388072e-04  2.85755268e-02]]


In [16]:
np.savetxt('Data/Covariance/PR_Cov_linTdep.csv',cov,delimiter=',')

## Apply Parameters + Plot

In [17]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [18]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.properties = GenericParameterBlock(default=configuration)

m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m.fs.F101.inlet.flow_mol.fix(1)
m.fs.F101.inlet.temperature.fix(323.15)
m.fs.F101.inlet.pressure.fix(399800)
m.fs.F101.inlet.mole_frac_comp[0,'R125'].fix(0.5)

m.fs.properties.PR_kappa_A['R125','emimTf2N'] = parameters['fs.properties.PR_kappa_A[R125,emimTf2N]']
m.fs.properties.PR_kappa_A['emimTf2N','R125'] = parameters['fs.properties.PR_kappa_A[emimTf2N,R125]']
m.fs.properties.PR_kappa_B['R125','emimTf2N'] = parameters['fs.properties.PR_kappa_B[R125,emimTf2N]']
m.fs.properties.PR_kappa_B['emimTf2N','R125'] = parameters['fs.properties.PR_kappa_B[emimTf2N,R125]']

# Initialize the flash unit
m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)

# Fix the state variables on the state block
# m.fs.F101.inlet.pressure.unfix()
m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].unfix()

R125_x = np.zeros((len(data_subset)))
P = np.zeros((len(data_subset)))
T = np.zeros((len(data_subset)))

m.fs.liq = Param(mutable=True,default=0.040)
m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, "emimTf2N"] == m.fs.liq)

for i in range(len(data_subset)):
    print('i:',i)
    print('Try x_IL:',data_subset["x_emimTf2N"].iloc[i])
    m.fs.liq = data_subset["x_emimTf2N"].iloc[i]
    m.fs.F101.inlet.temperature.fix(float(data_subset["temperature"].iloc[i]))
    m.fs.F101.inlet.pressure.fix(float(data_subset["pressure"].iloc[i]))
    m.fs.F101.inlet.mole_frac_comp[0,'R125'].fix(float(data_subset["x_R125"].iloc[i])+0.1)
    m.fs.F101.inlet.mole_frac_comp[0,'emimTf2N'].fix(float(1-(data_subset["x_R125"].iloc[i]+0.1)))
    m.fs.F101.vap_outlet.temperature.fix(float(data_subset["temperature"].iloc[i]))
    
    try:
        m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
        
    except:
        print('Initialization Error')
        
    try:

        status = solver.solve(m, tee = False)
        
        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):
            print('Feasible/Optimal Solution')
            R125_x[i] = value(m.fs.F101.liq_outlet.mole_frac_comp[0,'R125']) 
            P[i] = value(m.fs.F101.vap_outlet.pressure[0])
            T[i] = data_subset["temperature"].iloc[i]
        else:
            print('Infeasible Solution')
            R125_x[i] = np.nan
            P[i] = np.nan
            T[i] = data_subset["temperature"].iloc[i]
            
    except ValueError:
        R125_x[i] = np.nan
        P[i] = np.nan
        T[i] = data_subset["temperature"].iloc[i]
        print('Solver Error')

i: 0
Try x_IL: 0.915
Feasible/Optimal Solution
i: 1
Try x_IL: 0.832
Feasible/Optimal Solution
i: 2
Try x_IL: 0.75
Feasible/Optimal Solution
i: 3
Try x_IL: 0.669
Feasible/Optimal Solution
i: 4
Try x_IL: 0.587
Feasible/Optimal Solution
i: 5
Try x_IL: 0.504
Feasible/Optimal Solution
i: 6
Try x_IL: 0.417
Feasible/Optimal Solution
i: 7
Try x_IL: 0.319
Feasible/Optimal Solution
i: 8
Try x_IL: 0.945
Feasible/Optimal Solution
i: 9
Try x_IL: 0.891
Feasible/Optimal Solution
i: 10
Try x_IL: 0.839
Feasible/Optimal Solution
i: 11
Try x_IL: 0.787
Feasible/Optimal Solution
i: 12
Try x_IL: 0.733
Feasible/Optimal Solution
i: 13
Try x_IL: 0.683
Feasible/Optimal Solution
i: 14
Try x_IL: 0.578
Feasible/Optimal Solution
i: 15
Try x_IL: 0.479
Feasible/Optimal Solution
i: 16
Try x_IL: 0.97
Feasible/Optimal Solution
i: 17
Try x_IL: 0.941
Feasible/Optimal Solution
i: 18
Try x_IL: 0.912
Feasible/Optimal Solution
i: 19
Try x_IL: 0.883
Feasible/Optimal Solution
i: 20
Try x_IL: 0.848
Feasible/Optimal Solution
i: 2

In [19]:
fitted_data = pd.DataFrame(data=[R125_x,P/1e6,T])
fitted_data = fitted_data.T
fitted_data.columns = ['R125_x','P, MPa','T, K']
fitted_data.to_csv('Data/Fits/PR_FittedData_linTdep_subset.csv')

In [ ]:
# fitted_data = pd.DataFrame(data=[R125_x,P/1e6,T])
# fitted_data = fitted_data.T
# fitted_data.columns = ['R125_x','P, MPa','T, K']
# fitted_data.to_csv('Data/Fits/PR_FittedData_linTdep.csv')

In [ ]:
plt.plot(fitted_data.where(fitted_data['T, K']==283.1)['R125_x'],fitted_data.where(fitted_data['T, K']==283.1)['P, MPa'],"b-",label='T=283 K')
plt.plot(fitted_data.where(fitted_data['T, K']==298.1)['R125_x'],fitted_data.where(fitted_data['T, K']==298.1)['P, MPa'],"r-",label='T=298 K')
plt.plot(fitted_data.where(fitted_data['T, K']==323.1)['R125_x'],fitted_data.where(fitted_data['T, K']==323.1)['P, MPa'],"c-",label='T=323 K')
plt.plot(fitted_data.where(fitted_data['T, K']==348.1)['R125_x'],fitted_data.where(fitted_data['T, K']==348.1)['P, MPa'],"m-",label='T=348 K')
plt.plot(data_full["x_R125"],data_full["pressure"]/1e6,"*",color='orange',label="data")  
plt.title('Isotherm R125-[emim][TF2N]')
plt.ylabel('Pressure (MPa)')
plt.xlabel('x R125')
plt.grid(True)
plt.ylim(0,1.110000)
plt.legend()
plt.show()

### Print Eigvals, Eigvectors

An identifiability analysis can be performed with these as well as a FIM because the covariance matrix is the inverse of the FIM. That means that if there are any eigvals > 0, their eigvecs indicate directions of un-identifiability.

In [ ]:
eigvals, eigvecs = np.linalg.eig(cov)

In [ ]:
eigvals

In [ ]:
eigvecs

In [ ]:
np.savetxt('Data/EigenAnalysis/PR_vals_linTdep.csv',eigvals,delimiter=',')
np.savetxt('Data/EigenAnalysis/PR_vecs_linTdep.csv',eigvecs,delimiter=',')

In [ ]:
FIM = np.linalg.inv(cov) 

#A_opt
FIM_trace = np.trace(FIM)
cov_trace = np.trace(cov)

#D_opt
FIM_det = np.linalg.det(FIM)

#E_opt
eigvals_FIM, eigvecs_FIM = np.linalg.eig(FIM)
eigvals_FIM_min = eigvals_FIM.min()
eigvals_FIM_max = eigvals_FIM.max()

np.savetxt('Data/EigenAnalysis/PR_FIMtrdet_linTdep.csv',[cov_trace,FIM_trace,FIM_det,eigvals_FIM_min,eigvals_FIM_max])
np.savetxt('Data/EigenAnalysis/PR_FIMvals_linTdep.csv',eigvals_FIM,delimiter=',')
np.savetxt('Data/EigenAnalysis/PR_FIMvecs_linTdep.csv',eigvecs_FIM,delimiter=',')

## AIC Value

See: https://doi.org/10.1080/21642583.2018.1496042

AIC(k) = Nln(MSE(k))+2k

N: number of data points

MSE(k): mean squared error of the model with k parameters

k: number of parameters

MSE(k) = SSE(k)/N

SSE: sum of squared error of the model with k parameters

In [ ]:
#number of datapoints
N = len(data_subset)
#MSE(k)
MSE = obj_value/N
#number of parameters
k = 4

In [ ]:
AIC = N*np.log(MSE) + 2*k
AIC

In [ ]:
AIC_data = pd.DataFrame(data=[N,MSE,k,obj_value,AIC])
AIC_data = AIC_data.T
AIC_data.columns = ['N','MSE','k','Objective, Pa^2','AIC']
AIC_data.to_csv('Data/AIC/PR_AIC_linTdep.csv')